### Tính vi phân tự động

- Vi phân là phép tính thiết yếu trong hầu như tất cả thuật toán học sâu. Mặc dù các phép toán khá trực quan nhưng với các mô hình phức tạp thì việc tự tính rõ ràng rất dễ sai.
- Gói thư viện autograd giải quyết vấn đề này một cách nhanh chóng và hiệu quả bằng cách tự động hóa các phép dịch đạo hàm.
- Khi đưa dữ liệu chạy qua mô hình, autograd xây dựng một đồ thị và theo dõi xem dữ liệu nào kết hợp với các phép tính nào để tạo ra kết quả. Với đồ thị này autograd sau đó có thể lan truyền ngược gradient lại theo ý muốn.
- Lan truyền ngược ở đây chỉ là truy ngược lại đồ thị tính toán và điền vào đó các giá trị đạo hàm riêng theo từng tham số.

In [22]:
from mxnet import autograd, np, npx
npx.set_np()

x = np.arange(4)
x

array([0., 1., 2., 3.])

Lấy ví dụ ta cần tính vi phân của hàm số u = 2xT * x theo vector cột x.
## Lưu ý
1. Trước khi có thể tính gradient của y theo x ta cần nơi lưu trữ, và KHÔNG được cấp phát thêm bộ nhớ mỗi khi tính đạo hàm theo một biến xác định vì ta thường cập nhật cùng một tham số hàng vạn lần và sẽ nhanh chóng hết bộ nhớ.
2. Bản thân giá trị gradient theo vector x cũng là một vector với cùng kích thước. Do vậy trong mã nguồn sẽ trực quan hơn nếu ta lưu giá trị gradient tính theo x dưới dạng một thuộc tính của ndarray. Ta cấp bộ nhớ cho gradient của một ndarray bằng cách gọi phương thức attach_grad

In [23]:
x.attach_grad()

Sau khi tính toán gradient theo biến x, ta có thể truy cập nó thông qua thuộc tính grad. Để an toàn, x.grad được khởi tạo là một mảng chứa các giá trị 0. 
Điều này hợp lý vì trong học sâu, lấy gradient thường là để cập nhật các tham số bằng cách cộng hoặc trừ gradient để cực đại hoặc cực tiểu hóa hàm đó. Bằng cách khởi tạo gradent bằng mảng chứa giá trị 0, ta đảm bảo rằng bất kỳ cập nhật vô tình nào trước khi gradient được tính toán sẽ không làm thay đổi các giá trị của các tham số.

In [24]:
x.grad

array([0., 0., 0., 0.])

MXNet sẽ bật một thiết bị ghi hình để ghi lại đường đi của mỗi biến được tạo, và điều này chỉ xảy ra khi được ra lệnh rõ ràng.

In [25]:
with autograd.record():
    y = 2 * np.dot(x, x)
y

array(28.)

Do x là một ndarray có độ dài bằng 4, np.dot sẽ tính toán tích vô hướng của x và x, trả về một số vô hướng rồi gán cho y. Tiếp theo, ta sẽ tính toán gradient của y theo mỗi thành phần của x một cách tự động bằng cách gọi hàm backward của y.

In [26]:
y.backward()

Khi kiểm tra lại giá trị của x.grad, ta sẽ thấy nó được ghi đè bằng gradient mới được tính toán

In [27]:
x.grad

array([ 0.,  4.,  8., 12.])

Gradient tương ứng của hàm y = 2 * Transpose(x) * x là 4x.

In [28]:
x.grad == 4 * x

array([ True,  True,  True,  True])

Nếu ta tiếp tục tính gradient của một biến khác mà giá trị của nó là kết quả theo biến x thì nội dung trong x.grad sẽ bị ghi đè.

In [29]:
with autograd.record():
    y = x.sum()
y.backward()
x.grad

array([1., 1., 1., 1.])

### Truyền ngược cho các biến không phải số vô hướng
- Khi y không phải số vô hướng thì vi phân của một vector y theo vector x là một ma trận.
- Tuy nhiên, khi đối tượng này xuất hiện trong học sâu thì khi gọi làn truyền ngược trên một vector, ta đang tính toán hàm mất mát theo mỗi batch bao gồm một vài mẫu huấn luyện. Ở đây, ý định của ta không phải là tính toán ma trận vi phân mà là tính tổng của các đạo hàm riêng được tính toán độc lập cho mỗi mẫu trong batch,
- Vì vậy, khi ta gọi backward lên một biến vector y - là một hàm của x, MXNet sẽ cho rằng ta muốn tính tổng của gradient. Tức là MXNet sẽ tạo một biến mới có giá trị là số vô hướng bằng cách cộng lại các phần tử trong y và tính gradient theo x của biến mới này.

In [30]:
with autograd.record():
    y = x * x # Y là một vector có số dạng tương tự với X
y.backward()

u = x.copy()
u.attach_grad()

with autograd.record():
    v = (u * u).sum() # V là một biến vô hướng, V = Y.sum()
v.backward()

x.grad == u.grad

array([ True,  True,  True,  True])